In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 12 04:03:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
! pip install textattack datasets transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 17.7 MB/s 
     |████████████████████████████████| 451 kB 72.5 MB/s 
     |████████████████████████████████| 5.8 MB 50.6 MB/s 
     |████████████████████████████████| 72 kB 1.7 MB/s 
     |████████████████████████████████| 401 kB 80.6 MB/s 
     |████████████████████████████████| 365 kB 105.7 MB/s 
     |████████████████████████████████| 125 kB 87.8 MB/s 
     |████████████████████████████████| 769 kB 102.3 MB/s 
     |████████████████████████████████| 60 kB 9.3 MB/s 
     |████████████████████████████████| 3.6 MB 76.0 MB/s 
     |████████████████████████████████| 41.4 MB 284 kB/s 
     |████████████████████████████████| 132 kB 103.7 MB/s 
     |████████████████████████████████| 212 kB 102.8 MB/s 
     |████████████████████████████████| 182 kB 98.0 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |███████████████████████████

In [4]:
import pandas as pd
from datasets import list_datasets, list_metrics, load_dataset, get_dataset_config_names, load_metric, load_dataset_builder, get_dataset_split_names
import transformers
from transformers import  BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader



In [5]:
torch.cuda.is_available() 

True

In [6]:
from transformers import get_scheduler
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import numpy as np
import evaluate
from transformers import AutoTokenizer
from datasets import Dataset as HG_Dataset


# Clean Data Preprocessing 

In [7]:
torch.cuda.empty_cache()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def tokenize_function(examples):
    return model_tokenizer(examples["text"], 
                          padding="max_length",
                          truncation=True)

In [10]:
root = "/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project"

In [28]:
checkpoint_names = [
    "textattack/bert-base-uncased-imdb",
    "textattack/distilbert-base-uncased-imdb",
    "textattack/albert-base-v2-imdb",
    f"{root}/adv_trained_model/best_model"
    ]

In [22]:
df1 = pd.read_csv(f"{root}/data/log_BAEGarg2019_attack_BERT_1.csv")
df2 = pd.read_csv(f"{root}/data/log_BAEGarg2019_attack_BERT_2.csv")
all_BAE_attack_on_BERT = pd.concat([df1,df2])

In [23]:
all_BAE_attack_on_BERT.to_csv(f"{root}/data/log_BAEGarg2019_attack_BERT_all.csv")

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
dataset_names = ["log_A2TYoo2021_0.05",
                  "log_A2TYoo2021_0.1",
                  "log_A2TYoo2021_0.15",
                  "log_TextFoolerJin2019",
                 "log_BAEGarg2019",
                 "log_BAEGarg2019_attack_BERT_all"]

In [25]:
import pandas as pd

In [26]:
HG_datasets = {}
dfs = {}

for dataset_name in dataset_names:
  df = pd.read_csv(f"{root}/data/{dataset_name}.csv")
  _dataset = df[["perturbed_text","ground_truth_output"]]
  dfs[dataset_name] = _dataset

  _dataset = _dataset.rename({"perturbed_text":"text", 
                              "ground_truth_output":"labels"}
                             ,axis=1)
  
  dataset = HG_Dataset.from_pandas(_dataset)
  HG_datasets[dataset_name] = dataset

In [27]:
for name, df in dfs.items():
  print(f"name: {df.ground_truth_output.value_counts()}")

name: 1    3000
0    3000
Name: ground_truth_output, dtype: int64
name: 1    3000
0    3000
Name: ground_truth_output, dtype: int64
name: 1    2002
0    1998
Name: ground_truth_output, dtype: int64
name: 1    504
0    496
Name: ground_truth_output, dtype: int64
name: 1    504
0    496
Name: ground_truth_output, dtype: int64
name: 1    501
0    499
Name: ground_truth_output, dtype: int64


## textattack/bert-base-uncased-imdb

In [29]:
metric_results = {}

for hg_name in HG_datasets:
  for checkpoint_name in checkpoint_names:


    torch.cuda.empty_cache()
    dataset = HG_datasets[hg_name]
    print(checkpoint_name, hg_name)

    clf_metrics = evaluate.combine(["accuracy", "recall", "precision", "f1"])

    model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_name)
    model_tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_name)
    model.to(device)

    tokenized_datasets_all = dataset.map(
        lambda examples : model_tokenizer(examples["text"], 
                                  padding="max_length",
                                  truncation=True)
        )
    
    tokenized_datasets = tokenized_datasets_all.remove_columns(['text'])
    tokenized_datasets.set_format("torch")

    test_dataloader = DataLoader(tokenized_datasets,
                              shuffle=True, 
                              batch_size=8)

    model.eval()
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        clf_metrics.add_batch(predictions=predictions, references=batch["labels"])

    m_res = clf_metrics.compute()
    print(m_res)

    metric_results[(hg_name,checkpoint_name)] = m_res
    print("\n \n ")


textattack/bert-base-uncased-imdb log_A2TYoo2021_0.05


  0%|          | 0/6000 [00:00<?, ?ex/s]

{'accuracy': 0.8575, 'recall': 0.805, 'precision': 0.8994413407821229, 'f1': 0.8496042216358839}

 
 
textattack/distilbert-base-uncased-imdb log_A2TYoo2021_0.05


Downloading:   0%|          | 0.00/485 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

{'accuracy': 0.802, 'recall': 0.7584830339321357, 'precision': 0.8315098468271335, 'f1': 0.7933194154488518}

 
 
textattack/albert-base-v2-imdb log_BAEGarg2019_attack_BERT_all


  0%|          | 0/1000 [00:00<?, ?ex/s]

{'accuracy': 0.84, 'recall': 0.8243512974051896, 'precision': 0.8515463917525773, 'f1': 0.8377281947261663}

 
 
/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project/adv_trained_model/best_model log_BAEGarg2019_attack_BERT_all


  0%|          | 0/1000 [00:00<?, ?ex/s]

{'accuracy': 0.819, 'recall': 0.810379241516966, 'precision': 0.8252032520325203, 'f1': 0.8177240684793554}

 
 


In [ ]:
hg_name = "log_A2TYoo2021_0.05"
dataset = HG_datasets[hg_name]
checkpoint_name = "textattack/bert-base-uncased-imdb"

torch.cuda.empty_cache()

print(checkpoint_name, hg_name)

clf_metrics = evaluate.combine(["accuracy", "recall", "precision", "f1"])

model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_name)
model_tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_name)
model.to(device)

tokenized_datasets_all = dataset.map(tokenize_function)
tokenized_datasets = tokenized_datasets_all.remove_columns(['text'])
tokenized_datasets.set_format("torch")

test_dataloader = DataLoader(tokenized_datasets,
                          shuffle=True, 
                          batch_size=8)

model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    clf_metrics.add_batch(predictions=predictions, references=batch["labels"])

print(clf_metrics.compute())


textattack/bert-base-uncased-imdb log_A2TYoo2021_0.05


Downloading:   0%|          | 0.00/511 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/6000 [00:00<?, ?ex/s]

{'accuracy': 0.8575, 'recall': 0.805, 'precision': 0.8994413407821229, 'f1': 0.8496042216358839}


In [32]:
pd.DataFrame(metric_results).to_csv(f"{root}/metric_results.csv")

In [33]:
root

'/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project'